In [26]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool
from dotenv import load_dotenv
import os
from openai.types.responses import ResponseTextDeltaEvent
load_dotenv()


True

In [27]:
@function_tool
def how_many_jokes() -> int:
    return 1

In [28]:
from pydantic import BaseModel, Field

class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline of the joke")


In [51]:
async def test():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        output_type=Joke,
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            # print("Event delta")
            print(event.data.delta, end="", flush=True)
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print(f"-- Tool called: {event.item.raw_item.name}")
                print(f"-- Tool args: {event.item.raw_item.arguments}")
                print(f"-- Tool kwargs: {event.item}")
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
                output = ItemHelpers.text_message_output(event.item)
                parse_output = Joke.model_validate_json(output)
                print(f"Joke setup: {parse_output.setup}")
                print(f"Joke punchline: {parse_output.punchline}")
            else:
                pass  # Ignore other event types
        else:
            print(f"Unknown event type: {event.type}")

    print("=== Run complete ===")

In [52]:
output = await test()
print(output)

=== Run starting ===
Agent updated: Joker
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
Unknown event type: raw_response_event
-- Tool called: how_many_jokes
-- Tool args: {}
-- Tool kwargs: ToolCallItem(agent=Agent(name='Joker', instructions='First call the `how_many_jokes` tool, then tell that many jokes.', handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[FunctionTool(name='how_many_jokes', description='', params_json_schema={'properties': {}, 'title': 'how_many_jokes_args', 'type': 'object', 

In [48]:
from textwrap import dedent

HOW_MANY_SEARCHES = 3
prompt = dedent("""
  You are a helpful research assistant. You will be given a task to solve as best you can.
  Given a query, come up with a set of web searches to perform to best answer the query.
  To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Queries:', and 'Observation:' sequences.
  At each step, in the 'Thought:' sequence, you should first explain your reasoning towards the web search.
  Then create a few web searches queries 'Queries' to perform to best answer the query.
  Then 'Observation:' to identify if the web searches would provide the information needed to answer the query.
  Here are a few examples:
  ---
  Task: "Which city has the highest population: Guangzhou or Shanghai?"
  Thought: "I need to get the populations for both cities and compare them."
  Queries: ['What is the population of Guangzhou', 'What is the population of Shanghai' ]
  Observation: These queries are enough to answer the question.
  ---
  Task: "What is the current age of the pope, raised to the power 0.36?"
  Thought: "I need to find the age of the pope"
  Queries: ["How is the current pope", "How old is the current pope"]
  Observation: These queries are enough to answer the question, regarding the math calculation that is outside of my reach.

  Here are the rules you should always follow to solve your task:
  1. Always provide a 'Thought:' sequence, and a 'Queries:', 'Observation'.
  2. Only create the queries that will help with the search, you do not perform the search.
  3. Identify all the probable search queries that are relavant for the user query.
  4. Output {HOW_MANY_SEARCHES} terms to query for.

"""
).format(HOW_MANY_SEARCHES=HOW_MANY_SEARCHES)
print(prompt)



You are a helpful research assistant. You will be given a task to solve as best you can.
Given a query, come up with a set of web searches to perform to best answer the query.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Queries:', and 'Observation:' sequences.
At each step, in the 'Thought:' sequence, you should first explain your reasoning towards the web search.
Then create a few web searches queries 'Queries' to perform to best answer the query.
Then 'Observation:' to identify if the web searches would provide the information needed to answer the query.
Here are a few examples:
---
Task: "Which city has the highest population: Guangzhou or Shanghai?"
Thought: "I need to get the populations for both cities and compare them."
Queries: ['What is the population of Guangzhou', 'What is the population of Shanghai' ]
Observation: These queries are enough to answer the question.
---
Task: "What is the current age of the pope, raised 